In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

c:\Users\SVT\anaconda3\envs\Pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- 데이터셋 불러오기
    - root = 학습/테스트 데이터 저장 경로
    - train = 학습용 또는 테스트용 데이터셋 여부 지정
    - download=True : root에 데이터가 없는 경우 인터넷에서 다운로드
    - transform 과 target_transform은 특징(feature)과 정답(label) 변형(transform) 지정

In [3]:
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

26422272it [01:19, 333472.84it/s]                               


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



29696it [00:00, 109139.51it/s]                          


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



4422656it [00:03, 1301682.69it/s]                             


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



6144it [00:00, ?it/s]                   

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



- 파일에서 사용자 정의 데이터셋 만들기
    - ashionMNIST 이미지들은 img_dir 디렉토리에 저장되고, 정답은 annotations_file csv 파일에 별도로 저장

In [4]:
import os
import pandas as pd
from torchvision.io import read_image

In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):       # Dataset 객체 생성 시 한 번만 실행
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = target_transform

    def __len__(self):      # 데이터셋 샘플 개수 반환
        return len(self.img_labels)
  
    def __getitem__(self, idx):     # 주어진 인덱스 idx에 해당하는 샘플을 데이터셋에 불러오고 반환
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label    
    # 인덱스를 기반으로 디스크에서 이미지 위치 식별 후 read_image를 사용하여 이미지 텐서로 변환
    # self.img_labels의 csv데이터로부터 해당하는 정답(label)을 가져오고, 변환(transform)함수들은 직접 호출한 뒤,
    # 텐서 이미지와 라벨을 Python 사전(dict)형으로 반환  

- DataLoader로 학습용 데이터 준비
    - Dataset 은 데이터셋의 특징(feature)을 가져오고 하나의 샘플에 정답(label)을 지정하는 일을 한 번에 함
    - 모델을 학습할 때, 일반적으로 샘플ㄷ르을 미니배치로 전달하고 매 에포크마다 데이터를 다시 섞어서 과적합을 막고(overfit),
    - Python의 multiprocessiong 을 사용하여 데이터 검색 속도를 높이려 함
    - DataLoader는 간단한 API로 이러한 복잡한 과정들을 추상화한 순회 가능한 객체(iterabel)임

In [6]:
from torch.utils.data import DataLoader

trian_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

1. 신경망 모델 구성하기
    - FashionMNIST 데이터셋의 이미지들을 분류하는 신경망 구성

In [7]:
import os, torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


- 학습을 위한 장치 
    - torch.cuda 사용 가능하면 GPU, 그렇지 않으면 CPU

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


- 클래스 정의하기
    - 신경망 모델을 nn.Module의 하위클래스로 정의,
    - __init__에서 신경망 계층 초기화
    - nn.Module을 상속받은 모든 클래스는 forward 메소드에서 입력 데이터에 대한 연산 구현

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

- nn.Module
    - PyTorch의 nn라이브러리는 Neural Network의 모든 것을 포괄하는 모든 신경망 모델의 Base Class
    - nn.Module을 상속한 subclass가 신경망 모델로 사용되기 위해서는 앞서 소개한 두 메서드를 override해야함
- \__init__\(self): 내가 사용하고 싶은, 내 신경망 모델에 사용될 요소들을 정의 및 초기화 하는 메서드
- forward(self, x): __init__에서 정의된 구성품들을 연결하는 메서드

2. 모델 매개변수 최적화 하기
    - 에프크라고 부르는 각 반복단계에서 모델 출력을 추측하고, 추측과 정답 사이의 오류(손실)을 계산하고, 매개변수에 대한 오류의 도함수를 수집한 뒤, 경사하강법을 사용하여 파라미터들을 최적화 한다

In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [11]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

trian_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

- 하이퍼파라미터 : 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수
    - 에포크 수 : 데이터셋을 반복하는 횟수
    - 배치크기 : 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수 
    - 학습률 : 각 배치/에포크에서 모델의 매개변수를 조절하는 비율로, 값이 작을수록 학습 속도가 느려지고, 값이 커지면 학습 중 예측할 수 없는 동작 발생

- 최적화 단계(Optimization Loop) : 하이퍼파라미터를 설정한 뒤, 최적화 단계를 통해 모델을 학습하고 최적화할 수 있음, 최적화 단계의 각 반복(iteration)을 에포크라고 부름, 하나의 에폭은 두 부부으로 구성됨
    - 학습 단계(train_ loop) : 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴
    - 검증/테스트 단계(validation/test loop) : 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋 반복

- 손실 함수(loss function)
    - 일반적인 손심함수에는 회귀 문제에 사용하는 nn.MSELoss 나 분류에 사용하는 nn.NLLLoss, nn.CrossEntropyLoss등이 있다

In [12]:
# 손실함수 초기화
loss_fn = nn.CrossEntropyLoss()

In [15]:
# 학습률 하이퍼파라미터 정의
learning_rate = 1e-3

# 옵티마이저 : 학습하려는 모델의 매개변수와 학습률 하이퍼파라미터를 등록하여 옵티마이저 초기화
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

- 학습 단계(loop) 에서 최적화는 세 단계로 이루어진다
    - optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정
    - 기본적으로 변화도는 더해지기 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정

    - loss.backward()를 호출하여 예측선실을 역전파, PyTorch는 각 매개변수에 대한 손실의 변화도를 저장
    - 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수 조정

In [16]:
# 전체 구현
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측과 손실 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 ==0:
            loss, current = loss.item(), batch + len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Aug loss: {test_loss:>8f} \n")
    

- 손실함수와 옵티마이저를 초기화하고 train_loop 와 test_loop 에 전달함
- 모델의 성능 향상을 알아보기 위해 자유롭게 에폭 수를 증가시켜 볼 수 있음

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------")
    train_loop(trian_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Done")

Epoch 1
-----------------------
loss: 2.303689 [   64/60000]
loss: 2.296791 [  164/60000]
loss: 2.288880 [  264/60000]
loss: 2.293881 [  364/60000]
loss: 2.270637 [  464/60000]
loss: 2.260277 [  564/60000]
loss: 2.267838 [  664/60000]
loss: 2.249465 [  764/60000]
loss: 2.263236 [  864/60000]
loss: 2.242618 [  964/60000]
Test Error: 
 Accuracy: 35.2%, Aug loss: 2.235273 

Epoch 2
-----------------------
loss: 2.241686 [   64/60000]
loss: 2.230864 [  164/60000]
loss: 2.214506 [  264/60000]
loss: 2.241886 [  364/60000]
loss: 2.181225 [  464/60000]
loss: 2.157978 [  564/60000]
loss: 2.202582 [  664/60000]
loss: 2.153542 [  764/60000]
loss: 2.190974 [  864/60000]
loss: 2.145562 [  964/60000]
Test Error: 
 Accuracy: 39.2%, Aug loss: 2.128056 

Epoch 3
-----------------------
loss: 2.138322 [   64/60000]
loss: 2.113633 [  164/60000]
loss: 2.064516 [  264/60000]
loss: 2.129838 [  364/60000]
loss: 2.016926 [  464/60000]
loss: 1.955399 [  564/60000]
loss: 2.047356 [  664/60000]
loss: 1.942666 [ 

3. 모델 저장하고 불러오기

In [18]:
import torch
import torch.onnx as onnx
import torchvision.models as models

- 모델 가중치 저장하고 불러오기
    - PyTorch 모델은 학습한 매개변수를 state_dict라고 불리는 내부 상태 사전(internal state dictionary)에 저장함
    - 이 상태 값들은 torch.save 메서드를 사용하여 저장(persist)함

In [19]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\SVT/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:47<00:00, 11.7MB/s] 


- 모델 가중치를 불러오기 위해선, 먼저 동일한 모델의 인스턴스를 생성한 다음
- load_state_dict() 메서드를 사용하여 매개변수 불러옴

In [21]:
model = models.vgg16() # 기본 가중치를 불러오지 않으므로 pretrained=True를 지정하지 않음
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

- 모델의 형태를 포함하여 저장하고 불러오기
    - 모델의 가중치를 불러올 때, 신경망의 구조를 정의하기 위해 모델의 클래스를 먼저 생성해야했음
    - 이 클래스의 구조를 모델과 함께 저장하고 싶으면, (model.state_dict()가 아닌) model을 저장 함수에 전달

In [22]:
torch.save(model, 'model.pth')

In [23]:
# 모델 불러오기
model = torch.load('model.pth')